# 模型组网


模型组网是深度学习任务中的重要一环，该环节定义了神经网络的层次结构、数据从输入到输出的计算过程（即前向计算）等。

飞桨框架提供了多种模型组网方式，本文介绍如下几种常见用法：
* **直接使用内置模型**
* **使用 [paddle.nn.Sequential](https://www.paddlepaddle.org.cn/documentation/docs/zh/api/paddle/nn/Sequential_cn.html#sequential) 组网**
* **使用 [paddle.nn.Layer](https://www.paddlepaddle.org.cn/documentation/docs/zh/api/paddle/nn/Layer_cn.html#layer) 组网**
* **使用 [paddle.nn.LayerList](https://www.paddlepaddle.org.cn/documentation/docs/zh/api/paddle/nn/LayerList_cn.html#cn-api-fluid-dygraph-layerlist) 组网**

另外飞桨框架提供了 [paddle.summary](https://www.paddlepaddle.org.cn/documentation/docs/zh/api/paddle/summary_cn.html#summary) 函数方便查看网络结构、每层的输入输出 shape 和参数信息。

## 一、直接使用内置模型

飞桨框架目前在 [paddle.vision.models](https://www.paddlepaddle.org.cn/documentation/docs/zh/api/paddle/vision/Overview_cn.html#about-models) 下内置了计算机视觉领域的一些经典模型，只需一行代码即可完成网络构建和初始化，适合完成一些简单的深度学习任务，满足深度学习初阶用户感受模型的输入和输出形式、了解模型的性能。

In [1]:
import paddle

print('飞桨框架内置模型：', paddle.vision.models.__all__)

飞桨框架内置模型： ['ResNet', 'resnet18', 'resnet34', 'resnet50', 'resnet101', 'resnet152', 'VGG', 'vgg11', 'vgg13', 'vgg16', 'vgg19', 'MobileNetV1', 'mobilenet_v1', 'MobileNetV2', 'mobilenet_v2', 'LeNet']


以 LeNet 模型为例，可通过如下代码组网：

In [3]:
# 模型组网并初始化网络
lenet = paddle.vision.models.LeNet(num_classes=10)

# 可视化模型组网结构和参数
paddle.summary(lenet,(1, 1, 28, 28))

---------------------------------------------------------------------------
 Layer (type)       Input Shape          Output Shape         Param #    
   Conv2D-3       [[1, 1, 28, 28]]      [1, 6, 28, 28]          60       
    ReLU-3        [[1, 6, 28, 28]]      [1, 6, 28, 28]           0       
  MaxPool2D-3     [[1, 6, 28, 28]]      [1, 6, 14, 14]           0       
   Conv2D-4       [[1, 6, 14, 14]]     [1, 16, 10, 10]         2,416     
    ReLU-4       [[1, 16, 10, 10]]     [1, 16, 10, 10]           0       
  MaxPool2D-4    [[1, 16, 10, 10]]      [1, 16, 5, 5]            0       
   Linear-4          [[1, 400]]            [1, 120]           48,120     
   Linear-5          [[1, 120]]            [1, 84]            10,164     
   Linear-6          [[1, 84]]             [1, 10]              850      
Total params: 61,610
Trainable params: 61,610
Non-trainable params: 0
---------------------------------------------------------------------------
Input size (MB): 0.00
Forward/backward

{'total_params': 61610, 'trainable_params': 61610}

通过 [paddle.summary](https://www.paddlepaddle.org.cn/documentation/docs/zh/api/paddle/summary_cn.html#summary) 的结果可以看出，LeNet 模型包含 `Conv2D` 卷积层、`ReLU` 激活层、`MaxPool2D` 池化层以及 `Linear` 全连接层，这些层通过堆叠形成了 LeNet 模型，对应网络结构如下图所示。
<center><img src="images/lenet.png" width="600"></center>
<center><br>图1：LeNet网络结构示意图</br></center>

另外在 [paddle.summary](https://www.paddlepaddle.org.cn/documentation/docs/zh/api/paddle/summary_cn.html#summary) 的结果中可清晰地查看每一层的输入数据和输出数据的形状（Shape）、模型的参数量（Params）等信息，方便可视化地了解模型结构、分析数据计算和传递过程。



## 二、Paddle.nn 介绍

经典模型可以满足一些简单深度学习任务的需求，然后更多情况下，需要使用深度学习框架构建一个自己的神经网络，这时可以使用飞桨框架 [paddle.nn](https://www.paddlepaddle.org.cn/documentation/docs/zh/api/paddle/nn/Overview_cn.html) 下的 API 构建网络，该目录下定义了丰富的神经网络层和相关函数 API，如卷积网络相关的 Conv1D、Conv2D、Conv3D，循环神经网络相关的 RNN、LSTM、GRU 等，方便组网调用，详细清单可在 [API 文档](https://www.paddlepaddle.org.cn/documentation/docs/zh/api/paddle/nn/Overview_cn.html) 中查看。

飞桨提供继承类（class）的方式构建网络，并提供了几个基类，如：[paddle.nn.Sequential](https://www.paddlepaddle.org.cn/documentation/docs/zh/api/paddle/nn/Sequential_cn.html#sequential)、 
[paddle.nn.Layer](https://www.paddlepaddle.org.cn/documentation/docs/zh/api/paddle/nn/Layer_cn.html#layer)、[paddle.nn.LayerList](https://www.paddlepaddle.org.cn/documentation/docs/zh/api/paddle/nn/LayerList_cn.html#cn-api-fluid-dygraph-layerlist)，构建一个继承基类的子类，并在子类中添加子层（sublayers，如卷积层、全连接层等）可实现网络的构建，不同基类对应不同的组网方式：
 
* **使用 [paddle.nn.Sequential](https://www.paddlepaddle.org.cn/documentation/docs/zh/api/paddle/nn/Sequential_cn.html#sequential) 组网**：构建顺序的线性网络结构（如 LeNet、xxx）时，可以选择该方式。相比于 Layer 方式 ，Sequential 方式可以用更少的代码完成线性网络的构建。
* **使用 [paddle.nn.Layer](https://www.paddlepaddle.org.cn/documentation/docs/zh/api/paddle/nn/Layer_cn.html#layer) 组网（推荐）**：构建一些比较复杂的网络结构时，可以选择该方式。相比于 Sequential 方式，Layer 方式可以更灵活地组建各种网络结构。
* **使用 [paddle.nn.LayerList](https://www.paddlepaddle.org.cn/documentation/docs/zh/api/paddle/nn/LayerList_cn.html#cn-api-fluid-dygraph-layerlist) 组网**：xxxxxx




## 三、使用 paddle.nn.Sequential 组网


构建顺序的线性网络结构时，可以选择该方式，只需要按模型的结构顺序，一层一层加到 [paddle.nn.Sequential](https://www.paddlepaddle.org.cn/documentation/docs/zh/api/paddle/nn/Sequential_cn.html#sequential) 子类中即可。

参照前面图 1 所示的 LeNet 模型结构，构建该网络结构的代码如下：

In [4]:
from paddle import nn

# 使用 paddle.nn.Sequential 构建 LeNet 模型
lenet_Sequential = nn.Sequential(
    nn.Conv2D(1, 6, 3, stride=1, padding=1),
    nn.ReLU(),
    nn.MaxPool2D(2, 2),
    nn.Conv2D(6, 16, 5, stride=1, padding=0),
    nn.ReLU(),
    nn.MaxPool2D(2, 2),
    nn.Linear(400, 120),
    nn.Linear(120, 84), 
    nn.Linear(84, 10)
)
# 可视化模型组网结构和参数
paddle.summary(lenet_Sequential,(1, 1, 28, 28))

ValueError: (InvalidArgument) Input(Y) has error dim.Y'dims[0] must be equal to 5But received Y'dims[0] is 400
  [Hint: Expected y_dims[y_ndim - 2] == K, but received y_dims[y_ndim - 2]:400 != K:5.] (at /paddle/paddle/fluid/operators/matmul_v2_op.h:268)
  [operator < matmul_v2 > error]

以上代码实现的组网与 paddle.vision.models.LeNet 完全一样。

Sequential组网中框架做了什么：

使用Sequential组网方式的条件和限制：



## 四、使用 paddle.nn.Layer 组网

构建一些比较复杂的网络结构时，可以选择该方式，组网包括三个步骤：
1. 创建一个继承自 [paddle.nn.Layer](https://www.paddlepaddle.org.cn/documentation/docs/zh/api/paddle/nn/Layer_cn.html#layer) 的类；
1. 在类的构造函数 `__init__` 中定义组网用到的神经网络层（sublayer）；
1. 在类的前向计算函数 `forward` 中使用定义好的 sublayer 进行前向计算。

并且 sublayer 既可以通过基础的神经网络层 API（如卷积层、池化层、全连接层等）定义，也可以通过 nn.Sequential 或 nn.Layer 定义。由此可见，paddle.nn.Layer 的组网用法非常灵活，便于构建各种复杂网络。

仍然以 LeNet 模型为例，使用 paddle.nn.Layer 组网的代码如下：


In [3]:
# 使用 Subclass 方式构建 LeNet 模型
class LeNet(nn.Layer):
    def __init__(self, num_classes=10):
        super(LeNet, self).__init__()
        self.num_classes = num_classes
        self.features = nn.Sequential(
            nn.Conv2D(
                1, 6, 3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2D(2, 2),
            nn.Conv2D(
                6, 16, 5, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2D(2, 2))

        if num_classes > 0:
            self.fc = nn.Sequential(
                nn.Linear(400, 120),
                nn.Linear(120, 84), nn.Linear(84, num_classes))

    def forward(self, inputs):
        x = self.features(inputs)

        if self.num_classes > 0:
            x = paddle.flatten(x, 1)
            x = self.fc(x)
        return x
lenet_SubClass = LeNet()

# 可视化模型组网结构和参数
params_info = paddle.summary(lenet_SubClass,(1, 1, 28, 28))
print(params_info)

---------------------------------------------------------------------------
 Layer (type)       Input Shape          Output Shape         Param #    
   Conv2D-5       [[1, 1, 28, 28]]      [1, 6, 28, 28]          60       
    ReLU-5        [[1, 6, 28, 28]]      [1, 6, 28, 28]           0       
  MaxPool2D-5     [[1, 6, 28, 28]]      [1, 6, 14, 14]           0       
   Conv2D-6       [[1, 6, 14, 14]]     [1, 16, 10, 10]         2,416     
    ReLU-6       [[1, 16, 10, 10]]     [1, 16, 10, 10]           0       
  MaxPool2D-6    [[1, 16, 10, 10]]      [1, 16, 5, 5]            0       
   Linear-7          [[1, 400]]            [1, 120]           48,120     
   Linear-8          [[1, 120]]            [1, 84]            10,164     
   Linear-9          [[1, 84]]             [1, 10]              850      
Total params: 61,610
Trainable params: 61,610
Non-trainable params: 0
---------------------------------------------------------------------------
Input size (MB): 0.00
Forward/backward

## 五、使用 paddle.nn.LayerList 组网

待补充

# 六、总结

待补充

# 扩展阅读：模型的层（Layer）

选几个经典的层解读一下








# 扩展阅读：模型的参数（Parameter）

补充parameters的介绍
